- 75% data

## Reading Data and Importing Libraries ##

In [1]:
# setting debug or not
check_test = False # check_testの場合、推論結果を最後にall_testオブジェクトで出力。（sub時はFalse）
use_train = False # Trueの場合trainの最初の1000行を使って推論 (sub時はFalse)
del_test_dict = False # Trueの場合、trainを使うとき、指定列（下で指定）の辞書を初期化した結果を出力する（学習用DMとの比較を可能にする）（sub時はFalse）

if del_test_dict:
    # にのぴーfeatureの辞書を初期化して、trainと同等の結果が出てくるようにする
    fill_nan_cols = []
        
    #fill_0_cols = ['un_acs_u_p_wt_same_que', 'un_acs_u_p_per_same_que']

In [2]:
import sys
import pandas as pd
import pickle as pkl
import numpy as np
import ast
import time
import gc
import scipy as sp
from collections import defaultdict
from sklearn.metrics import roc_auc_score

import warnings
warnings.simplefilter('ignore') 

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)
pd.options.display.float_format = '{:.2f}'.format

In [3]:
import sys
sys.path.append("/kaggle/input/riiid-idem-part")

from ide_nn.src.share import (
    load_params_MLP,
#     load_params_TabNet,
    load_params_Linear,
    load_FeatureTransformer,
    load_MLP,
#     load_TabNet,
    load_Linear,
    predict_MLP,
#     predict_TabNet,
    predict_Linear,
)


mlp_params = load_params_MLP(src_dir="/kaggle/input/riiid-idem-part")
mlp_ft = load_FeatureTransformer(mlp_params)
mlp_model = load_MLP(mlp_params, mlp_ft)

## read data/model

## make content master
- content_idとlecture_idをまとめた形に加工
- lectureとcontent_idが被っている場合、partはcontentの方を採用する

In [4]:
#read base dataset
questions = pd.read_csv('../input/riiid-test-answer-prediction/questions.csv')
questions['L_R'] = np.where(questions['part'] < 5, 1, 0)
lectures = pd.read_csv('../input/riiid-test-answer-prediction/lectures.csv')

#read tag master
tag_master = pd.read_pickle('../input/riiid-feature/tag_master.pickle')
content_diff_master = pd.read_pickle('../input/riiid-feature/content_diff_master.pickle')

# read content features
content_elp_accuracy = pd.read_pickle('../input/riiid-feature/content_elp_accuracy.pickle')
content_elp_count_ratio = pd.read_pickle('../input/riiid-feature/content_elp_count_ratio.pickle')
content_lag_accuracy = pd.read_pickle('../input/riiid-feature/content_lag_accuracy.pickle')
content_lag_count_ratio = pd.read_pickle('../input/riiid-feature/content_lag_count_ratio.pickle')
content_count_ratio = pd.read_pickle('../input/riiid-feature/content_count_ratio_df.pickle')
content_accuracy = pd.read_pickle('../input/riiid-feature/content_accuracy_df.pickle')
content_pivot = pd.read_pickle('../input/riiid-feature/bin_pivot_df.pickle')
content_dup_ratio = pd.read_pickle('../input/riiid-feature/content_dup_ratio_df.pickle')
content_unacc_df = pd.read_csv('../input/riiid-feature/content_unacc_user_prior_user_answered_correctly.csv')

In [5]:
# content * lagtime_bin
content_same_lag_accuracy = pd.read_pickle('../input/riiid-feature/content_accu_lag.pickle')

# content * accu_bin
content_same_prioraccu_accuracy = pd.read_pickle('../input/riiid-feature/content_same_user_accu.pickle')

In [6]:
# add L/R tag difficult features
questions.rename(columns = {'question_id':'content_id'}, inplace = True)
questions['L_R'] = np.where(questions['part'] < 5, 1, 0)
questions = questions.merge(tag_master[['content_id', 'tag']], on = 'content_id', how = 'left')
questions = questions.merge(content_diff_master, on = 'content_id', how = 'left')

# contentとlectureを縦に結合し、欠損埋め
lectures.rename(columns = {'lecture_id':'content_id'}, inplace = True)
questions_all = pd.concat([questions[['content_id']], lectures[['content_id']]],axis=0).drop_duplicates().reset_index(drop=True)
questions_all = questions_all.merge(questions[['content_id', 'bundle_id', 'part', 'L_R', 'tag', 'prob_difficuly']], on = 'content_id', how = 'left')
questions_all['part'].fillna(99, inplace = True)
questions_all['L_R'].fillna(99, inplace = True)
questions_all['bundle_id'].fillna(99999, inplace = True)
questions_all['tag'].fillna(999, inplace = True)
questions = questions_all.copy()

In [7]:
del questions_all

## モデルと特徴量を読み込み

In [8]:
#read model/make dataset
model = pd.read_pickle('../input/riiid-model/full_model_2.pickle')

In [9]:
## read feature
# content
content_base= pd.read_pickle('/kaggle/input/riiid-feature/content_df_base.pickle')
content_df = pd.read_pickle('/kaggle/input/riiid-feature/content_df.pickle')
content_difficult_df = pd.read_pickle('../input/riiid-feature/content_difficult_pred.pickle')
answer_score_df = pd.read_pickle('../input/riiid-feature/answer_score_df.pickle')
w2v_10 = pd.read_pickle('../input/riiid-feature/df_accu_cont_10_w2v.pickle')
content_series = pd.read_pickle('../input/riiid-feature/content_series.pickle')
lag_content = pd.read_pickle('../input/riiid-feature/lag_content_df.pickle')
content_acc = pd.read_pickle('../input/riiid-feature/content_acc_considered_part.pickle')

#user
user_base = pd.read_pickle('../input/riiid-feature/user_answered_correctly_df.pickle')
user_base_part = pd.read_pickle('../input/riiid-feature/user_answered_correctly_df_part.pickle')
user_score = pd.read_pickle('../input/riiid-feature/user_score_df.pickle')
del user_score['count']
accu_df = pd.read_pickle('../input/riiid-feature/LR_df.pickle')
last_accu_df = pd.read_pickle('../input/riiid-feature/last_accu_cont.pickle')
last_content_lag_df = pd.read_pickle('../input/riiid-feature/last_content_lag_df.pickle')
before_lecture_df = pd.read_pickle('../input/riiid-feature/before_lecture_df.pickle')
before_lecture_df.fillna(0, inplace = True)
diff_task_df = pd.read_pickle('../input/riiid-feature/diff_task_pred_features.pickle')
dup_ratio_df = pd.read_pickle('../input/riiid-feature/dup_ratio_df.pickle')
dup_row_num_df = pd.read_pickle('../input/riiid-feature/pred_dup_row_num.pickle')
cont_difficulty_df = pd.read_pickle('../input/riiid-feature/pred_cont_difficulty.pickle')
user_score_LR = pd.read_pickle('../input/riiid-feature/pred_user_score_LR.pickle')
del user_score_LR['L_count'], user_score_LR['R_count']

last_lag_bin_df = pd.read_pickle('../input/riiid-feature/last_lag_bin_df.pickle')
last_elp_bin_df = pd.read_pickle('../input/riiid-feature/last_elp_bin_df.pickle')

# user&part
#user_unacc_df = pd.read_pickle('../input/riiid-feature/pred_cumsum_unaccu_df.pickle')
#user_unacc_df.fillna(0, inplace = True)

# time
first_user_df = pd.read_pickle('../input/riiid-feature/first_user_df.pickle')
lag_time_df = pd.read_pickle('../input/riiid-feature/last_time_stamp_w_start.pickle')
prior_time_df = pd.read_pickle('../input/riiid-feature/prior_time_df.pickle')
end_to_start_df = pd.read_pickle('../input/riiid-feature/last_end_to_start_df.pickle')
session_time_df = pd.read_pickle('../input/riiid-feature/session_time.pickle')

del session_time_df['session_last_accu']

In [10]:
# matrix

def pickle_load(path):
    with open(path, mode='rb') as f:
        data = pkl.load(f)
        return data

# user * content毎に前回のtimestampを入れたmatrix
dup_time_matrix = sp.sparse.load_npz('../input/add-matrix-data/dup_time_matrix.npz')
dup_time_matrix = dup_time_matrix.tolil()

# user * tag毎に今までの回答回数を入れたmatrix
tag_matrix = sp.sparse.load_npz('../input/add-matrix-data/tag_matrix.npz')
tag_matrix = tag_matrix.tolil()

# 各種master（idをmatrixの列番号に変換するマスタ）
user_master = pickle_load( '../input/add-matrix-data/user_master.pickle')
content_master = pickle_load( '../input/add-matrix-data/content_master.pickle')
tag_mat_master = pickle_load( '../input/add-matrix-data/tag_mat_master.pickle')

In [11]:
# rename
first_user_df.rename(columns = {'content_user_cover_ratio':'first_user_ratio'}, inplace = True)
lag_time_df.rename(columns = {'timestamp':'last_timestamp'}, inplace = True)
session_time_df.rename(columns = {'session_prob':'prior_session_prob'}, inplace = True)
lag_content.rename(columns = {'lag_time':'mean_lag_time'}, inplace = True)

## merge(user)

In [12]:
%%time
user_base = user_base.merge(user_base_part, on = 'user_id', how = 'left')
user_base = user_base.merge(accu_df, on = 'user_id', how = 'left')
user_base = user_base.merge(user_score, on = 'user_id', how = 'left')
user_base = user_base.merge(first_user_df, on = 'user_id', how = 'left') # 0.442076
user_base = user_base.merge(lag_time_df, on = 'user_id', how = 'left')
user_base = user_base.merge(content_difficult_df, on = 'user_id', how = 'left')
user_base = user_base.merge(before_lecture_df, on = 'user_id', how = 'left')
user_base = user_base.merge(last_accu_df, on = 'user_id', how = 'left')
user_base = user_base.merge(end_to_start_df, on = 'user_id', how = 'left')
user_base = user_base.merge(session_time_df, on = 'user_id', how = 'left')
user_base = user_base.merge(last_content_lag_df, on = 'user_id', how = 'left')
user_base = user_base.merge(diff_task_df, on = 'user_id', how = 'left')
user_base = user_base.merge(dup_ratio_df, on = 'user_id', how = 'left')
user_base = user_base.merge(dup_row_num_df, on = 'user_id', how = 'left')
user_base = user_base.merge(cont_difficulty_df, on = 'user_id', how = 'left')
user_base = user_base.merge(user_score_LR, on = 'user_id', how = 'left')
user_base = user_base.merge(last_lag_bin_df, on = 'user_id', how = 'left')
user_base = user_base.merge(last_elp_bin_df, on = 'user_id', how = 'left')
user_base['lag_time_bin'] = np.nan
user_base['elp_time_bin'] = np.nan
user_base['prior_user_answered_correctly_bin'] = np.nan
user_base['prior_answer'] = np.nan # elp_time系の変数

content_base = content_base.merge(content_df, on = 'content_id', how = 'left')
content_base = content_base.merge(w2v_10, on = 'content_id', how = 'left')
content_base = content_base.merge(content_series, on = 'content_id', how = 'left')
content_base = content_base.merge(lag_content, on = 'content_id', how = 'left')
content_base = content_base.merge(questions[['part', 'bundle_id']], left_on = 'content_id', right_index = True, how = 'left')
content_base = content_base.merge(prior_time_df, on = 'bundle_id', how = 'left')

content_base = content_base.merge(content_elp_accuracy, on = 'content_id', how = 'left')
content_base = content_base.merge(content_elp_count_ratio, on = 'content_id', how = 'left')
content_base = content_base.merge(content_lag_accuracy, on = 'content_id', how = 'left')
content_base = content_base.merge(content_lag_count_ratio, on = 'content_id', how = 'left')
content_base = content_base.merge(content_count_ratio, on = 'content_id', how = 'left')
content_base = content_base.merge(content_accuracy, on = 'content_id', how = 'left')
content_base = content_base.merge(content_pivot, on = 'content_id', how = 'left')
content_base = content_base.merge(content_dup_ratio, on = 'content_id', how = 'left')


# ninopy features
content_base = content_base.merge(content_acc, on = 'content_id', how = 'left')
content_base = content_base.merge(content_unacc_df, on = 'content_id', how = 'left')

content_base.loc[content_base['answered_correctly_content'] == 0, 'answered_correctly_content'] = content_base['answered_correctly_content'].mean()

for col in ['answered_correctly_content_first_look','fix_answered_correctly_content_first_look']:
    content_base[col].fillna(content_base[col].mean(), inplace = True)

del accu_df, content_difficult_df, before_lecture_df, prior_time_df, first_user_df

CPU times: user 2.02 s, sys: 109 ms, total: 2.13 s
Wall time: 2.18 s


In [13]:
# 確率系のfeature作成
user_base['prior_user_answered_correctly'] = user_base['answered_correctly_cumsum'] / user_base['count']
user_base['prior_L_accu'] = user_base['L_accu_sum'] / user_base['L_count']
user_base['prior_R_accu'] = user_base['R_accu_sum'] / user_base['R_count']
user_base['prior_user_score'] = user_base['user_score_cumsum'] / user_base['count']
user_base['prior_session_prob'] = user_base['prior_session_prob']
user_base['prior_L_user_score'] = user_base['L_score'] / user_base['L_count']
user_base['prior_R_user_score'] = user_base['R_score'] / user_base['R_count']
user_base['difficulty_50_ratio'] = user_base['difficulty_50'] / user_base['count_dif_50']
user_base['difficulty_75_ratio'] = user_base['difficulty_75'] / user_base['count_dif_75']
user_base['difficulty_1_ratio'] = user_base['difficulty_1'] / user_base['count_dif_1']
user_base['prior_p2_accu'] = user_base['p2_accu_sum'] / user_base['p2_count']
user_base['prior_p5_accu'] = user_base['p5_accu_sum'] / user_base['p5_count']

In [14]:
#user_unacc_df.head(10)

In [15]:
# debug用
if use_train:
    print('use_train')
    if del_test_dict:
        print('del_test_dict')
        for col in list(user_base.columns[1:]):
            user_base[col] = 0
            # 辞書のデフォにnanを入れる変数たち
            for col in ['un_acs_u_p_wt_same_que_avg', 'un_acs_u_p_per_same_que_avg',
                    'diff_un_acs_u_p_per_same_que_avg', 'diff_un_acs_u_p_wt_same_que_avg', 
                    'ratio_un_acs_u_p_per_same_que_avg','ratio_un_acs_u_p_wt_same_que_avg']:
                user_unacc_df[col] = np.nan
                
        for col in ['un_acs_u_p_wt_same_que', 'un_acs_u_p_per_same_que']:
            
            user_unacc_df[col] = 0
            

## make dict

In [16]:
#user_unacc_dict = user_unacc_df.set_index(['user_id', 'part']).to_dict(orient='dict')

In [17]:
%%time
# make dict
print('user_dict')
user_accu_dict = user_base.set_index(['user_id']).to_dict(orient='dict')
print('content_dict')
content_dict = content_df[['content_id', 'answered_correctly_content_first_look', 'content_user_cover_ratio']
                         ].set_index(['content_id']).to_dict(orient='dict')
print('user_content_dict')
answer_score_dict = answer_score_df.set_index(['content_id', 'user_answer']).to_dict(orient='dict')

print('same_lag_dict')
content_same_lag_accuracy_dict = content_same_lag_accuracy.set_index(['content_id', 'lag_time_bin']).to_dict(orient='dict')
content_same_prioraccu_accuracy_dict = content_same_prioraccu_accuracy.set_index(['content_id', 'prior_user_answered_correctly_bin']).to_dict(orient='dict')


del user_base, content_df, answer_score_df

user_dict
content_dict
user_content_dict
same_lag_dict
CPU times: user 14.5 s, sys: 1.62 s, total: 16.1 s
Wall time: 16.1 s


## set index

In [18]:
# dictとは別に紐づける特徴量（content系）にはindexを振る
questions = questions.set_index('content_id')
content_base = content_base.set_index('content_id')

# 推論メインパート

## デバック用のループ関数

In [19]:
class Iter_Valid(object):
    def __init__(self, df, max_user=1000):
        df = df.reset_index(drop=True)
        self.df = df
        self.user_answer = df['user_answer'].astype(str).values
        self.answered_correctly = df['answered_correctly'].astype(str).values
        df['prior_group_responses'] = "[]"
        df['prior_group_answers_correct'] = "[]"
        self.sample_df = df[df['content_type_id'] == 0][['row_id']]
        self.sample_df['answered_correctly'] = 0
        self.len = len(df)
        self.user_id = df.user_id.values
        self.task_container_id = df.task_container_id.values
        self.content_type_id = df.content_type_id.values
        self.max_user = max_user
        self.current = 0
        self.pre_user_answer_list = []
        self.pre_answered_correctly_list = []

    def __iter__(self):
        return self
    
    def fix_df(self, user_answer_list, answered_correctly_list, pre_start):
        df= self.df[pre_start:self.current].copy()
        sample_df = self.sample_df[pre_start:self.current].copy()
        df.loc[pre_start,'prior_group_responses'] = '[' + ",".join(self.pre_user_answer_list) + ']'
        df.loc[pre_start,'prior_group_answers_correct'] = '[' + ",".join(self.pre_answered_correctly_list) + ']'
        self.pre_user_answer_list = user_answer_list
        self.pre_answered_correctly_list = answered_correctly_list
        return df, sample_df

    def __next__(self):
        added_user = set()
        pre_start = self.current
        pre_added_user = -1
        pre_task_container_id = -1
        pre_content_type_id = -1
        user_answer_list = []
        answered_correctly_list = []
        while self.current < self.len:
            crr_user_id = self.user_id[self.current]
            crr_task_container_id = self.task_container_id[self.current]
            crr_content_type_id = self.content_type_id[self.current]
            if crr_user_id in added_user and (crr_user_id != pre_added_user or (crr_task_container_id != pre_task_container_id and crr_content_type_id == 0 and pre_content_type_id == 0)):
                
                # known user(not prev user or (differnt task container and both question))
                return self.fix_df(user_answer_list, answered_correctly_list, pre_start)
            if len(added_user) == self.max_user:
                if  crr_user_id == pre_added_user and (crr_task_container_id == pre_task_container_id or crr_content_type_id == 1):
                    user_answer_list.append(self.user_answer[self.current])
                    answered_correctly_list.append(self.answered_correctly[self.current])
                    self.current += 1
                    continue
                else:
                    return self.fix_df(user_answer_list, answered_correctly_list, pre_start)
            added_user.add(crr_user_id)
            pre_added_user = crr_user_id
            pre_task_container_id = crr_task_container_id
            pre_content_type_id = crr_content_type_id
            user_answer_list.append(self.user_answer[self.current])
            answered_correctly_list.append(self.answered_correctly[self.current])
            self.current += 1
        if pre_start < self.current:
            return self.fix_df(user_answer_list, answered_correctly_list, pre_start)
        else:
            raise StopIteration()

## bin系関数

In [20]:
def bin_lag_time(x):
    if x <= 30000:
        bin = 0
    elif x < 65000:
        bin = 1
    elif x < 170000:
        bin = 2
    else:
        bin = 3
    return bin


def bin_accuracy(x):
    if x <= 0.55:
        bin = 0
    elif x < 0.65:
        bin = 1
    elif x < 0.75:
        bin = 2
    elif x <= 1:
        bin = 3
    else:
        bin = 4 # np.nan
    return bin


def bin_elp_time(x):
    if x <= 16000:
        bin = 0
    elif x < 21000:
        bin = 1
    elif x < 30000:
        bin = 2
    elif x < 300000:
        bin = 3
    else:
        bin = 4 # np.nanと300000の枠
    return bin


## 使用変数の定義

In [21]:
use_col_lgb = [  
 'prior_question_elapsed_time',
 'lag_time',
 'prior_user_answered_correctly',
 'prior_user_score',
 'count',
 'count_part',
 'content_user_cover_ratio',
 'first_user_ratio',
 'answered_correctly_content',
 'answered_correctly_content_first_look',
 'fix_answered_correctly_content_first_look',
 'part',
 'before_lecture',
 'L_count', 
 'R_count', 
 'prior_L_accu', 
 'prior_R_accu',
 'content_accu_mean',
 'content_cumsum_ratio', 
 'ave_elapsed_time',
 'prior_accu_continuous',
 'prior_accu_no_continuous',
 'dup_time',
 'prior_end_to_start', 
 'content_series_mean',
 'tag',
 'prior_question_had_explanation',
 'lec_num',
 'session_time',
 'session_count',
 'prior_session_time',
 'prior_session_prob',
 'mean_lag_time',
 'lag_user_ave',
 'lag_user_p2_ave',
 'lag_user_p5_ave',
 'lag_c_p2',
 'lag_c_p5',
 'lag_task',
 'dup_ratio',
 'count_dif_50_ratio', 
 'count_dif_75_ratio', 
 'count_dif_1_ratio',
 'difficulty_50_ratio',
 'difficulty_75_ratio',
 'difficulty_1_ratio',
 'dup_row_num',

 'prior_L_user_score',
 'prior_R_user_score',
 'user_score_ratio',
 'prior_p2_accu',
 'prior_p5_accu',
    
 # lag_u_features
 'lag_u_bin_0_accu_mean',
 'lag_u_bin_1_accu_mean',
 'lag_u_bin_2_accu_mean',
 'lag_u_bin_3_accu_mean',
 'lag_u_bin_0_count_ratio',
 'lag_u_bin_1_count_ratio',
 'lag_u_bin_2_count_ratio',
 'lag_u_bin_3_count_ratio',
 'elp_u_bin_0_accu_mean',
 'elp_u_bin_1_accu_mean',
 'elp_u_bin_2_accu_mean',
 'elp_u_bin_3_accu_mean',
 'elp_u_bin_0_count_ratio',
 'elp_u_bin_1_count_ratio',
 'elp_u_bin_2_count_ratio',
 'elp_u_bin_3_count_ratio',
    
 # content * prior_accu
 'content_accuracy',
    
 # content * lag
 'same_lag_user_content_accuracy',
    
 # lag
 'content_lag_count_dist_0',
 'content_lag_count_dist_1',
 'content_lag_count_dist_2',
 'content_lag_count_dist_3',
 'same_lagbin_content_acc_mean_0',
 'same_lagbin_content_acc_mean_1',
 'same_lagbin_content_acc_mean_2',
 'same_lagbin_content_acc_mean_3',
    
 # elp
 'content_elp_count_dist_0',
 'content_elp_count_dist_1',
 'content_elp_count_dist_2',
 'content_elp_count_dist_3',
 'same_elpbin_content_acc_mean_0',
 'same_elpbin_content_acc_mean_1',
 'same_elpbin_content_acc_mean_2',
 'same_elpbin_content_acc_mean_3',
    
 # acc
 'content_bin_count_ratio_0',
 'content_bin_count_ratio_1',
 'content_bin_count_ratio_2',
 'content_bin_count_ratio_3',
 'same_prioracbin_content_acc_mean_0',
 'same_prioracbin_content_acc_mean_1',
 'same_prioracbin_content_acc_mean_2',
 'same_prioracbin_content_acc_mean_3',
    
 # ninopy
 'content_unacc_user_prior_user_answered_correctly_mean',
 'content_unacc_user_prior_user_answered_correctly_std',
    
 # additional content_id feature
 'content_dup_ratio', 
 'content_dup_count_ratio',
 'bin_0_accu',
 'bin_1_accu',
 'bin_2_accu',
 'bin_3_accu',
 'bin_4_accu',
    
] + ['vec_w2v_' + str(i) for i in range(0, 10)] 

In [22]:
use_col_mlp = [  
 'prior_question_elapsed_time',
 'lag_time',
 'prior_user_answered_correctly',
 'prior_user_score',
 'count',
 'count_part',
 'content_user_cover_ratio',
 'first_user_ratio',
 'answered_correctly_content',
 'answered_correctly_content_first_look',
 'fix_answered_correctly_content_first_look',
 'part',
 'before_lecture',
 'L_count', 
 'R_count', 
 'prior_L_accu', 
 'prior_R_accu',
 'content_accu_mean',
 'content_cumsum_ratio', 
 'ave_elapsed_time',
 'prior_accu_continuous',
 'prior_accu_no_continuous',
 'dup_time',
 'prior_end_to_start', 
 'content_series_mean',
 'tag',
 'prior_question_had_explanation',
 'lec_num',
 'session_time',
 'session_count',
 'prior_session_time',
 'prior_session_prob',
 'mean_lag_time',
 'lag_user_ave',
 'lag_user_p2_ave',
 'lag_user_p5_ave',
 'lag_c_p2',
 'lag_c_p5',
 'lag_task',
 'dup_ratio',
 'count_dif_50_ratio', 
 'count_dif_75_ratio', 
 'count_dif_1_ratio',
 'difficulty_50_ratio',
 'difficulty_75_ratio',
 'difficulty_1_ratio',
 'dup_row_num',

 'prior_L_user_score',
 'prior_R_user_score',
 'user_score_ratio',
 'prior_p2_accu',
 'prior_p5_accu',
    
 # lag_u_features
 'lag_u_bin_0_accu_mean',
 'lag_u_bin_1_accu_mean',
 'lag_u_bin_2_accu_mean',
 'lag_u_bin_3_accu_mean',
 'lag_u_bin_0_count_ratio',
 'lag_u_bin_1_count_ratio',
 'lag_u_bin_2_count_ratio',
 'lag_u_bin_3_count_ratio',
 'elp_u_bin_0_accu_mean',
 'elp_u_bin_1_accu_mean',
 'elp_u_bin_2_accu_mean',
 'elp_u_bin_3_accu_mean',
 'elp_u_bin_0_count_ratio',
 'elp_u_bin_1_count_ratio',
 'elp_u_bin_2_count_ratio',
 'elp_u_bin_3_count_ratio',
    
 # content * prior_accu
 'content_accuracy',
    
 # content * lag
 'same_lag_user_content_accuracy',
    
 # ninopy
 'content_unacc_user_prior_user_answered_correctly_mean',
 'content_unacc_user_prior_user_answered_correctly_std',
    
 # additional content_id feature
 'content_dup_ratio', 
 'content_dup_count_ratio',
 'bin_0_accu',
 'bin_1_accu',
 'bin_2_accu',
 'bin_3_accu',
 'bin_4_accu',

] + ['vec_w2v_' + str(i) for i in range(0, 10)] 

In [23]:
user_accu_dict.keys()

dict_keys(['count', 'answered_correctly_cumsum', 'count_part', 'answered_correctly_cumsum_part', 'L_count', 'R_count', 'L_accu_sum', 'R_accu_sum', 'user_score_cumsum', 'first_user_ratio', 'lag_time', 'last_timestamp', 'content_cumsum', 'before_lecture', 'prior_accu_continuous', 'prior_accu_no_continuous', 'last_end_to_start_time', 'last_question_elapsed_time', 'session_time', 'session_first_time', 'session_count', 'session_num', 'session_accu', 'prior_session_prob', 'prior_session_time', 'lag_user', 'lag_user_p2', 'lag_user_p5', 'lag_c', 'lag_c_p2', 'lag_c_p5', 'last_task_container_id', 'lag_task', 'dup_num_2', 'last_bundle_id', 'last_dup_row_num', 'count_dif_50', 'count_dif_75', 'count_dif_1', 'difficulty_50', 'difficulty_75', 'difficulty_1', 'L_score', 'R_score', 'p2_accu_sum', 'p5_accu_sum', 'p2_count', 'p5_count', 'lag_u_bin_0_accu_sum', 'lag_u_bin_1_accu_sum', 'lag_u_bin_2_accu_sum', 'lag_u_bin_3_accu_sum', 'lag_u_bin_0_accu_mean', 'lag_u_bin_1_accu_mean', 'lag_u_bin_2_accu_mean',

## sparse matrixを使っている特徴量の更新関数

In [24]:
def update_dup_num(test_df, user_accu_dict, dup_time_matrix, user_master, content_master):
    
    #----------------------------------
    # update dup_ratio and dup_time using sparse matrix
    #----------------------------------

    dup_ratio = np.zeros(len(test_df), dtype=np.float64)
    dup_time = np.zeros(len(test_df), dtype=np.float64)

    for cnt, row in enumerate(test_df[['user_id', 'content_id', 'content_type_id', 'timestamp']].values):
        
        user_accu_dict['dup_num_2'].setdefault(row[0], 0)

        # 各user/回答contentで既に時間が入っている場合、現timestamp - 過去回答時間を入力
        # user_idとcontent_idはuser_masterとcontent_master内でそれぞれ行列番号に変換
        if dup_time_matrix[user_master[row[0]], content_master[row[1]]] > 0:
            dup_time[cnt] = row[3] - (np.exp(dup_time_matrix[user_master[row[0]], content_master[row[1]]]) - 1)
            
            # 重複があるたびにカウントアップ
            user_accu_dict['dup_num_2'][row[0]] += 1
        else:
            dup_time[cnt] = 0
        
        # dup_ratio：重複のあった数/全回答数をユーザー毎にrollingして求める
        dup_ratio[cnt] = np.array(user_accu_dict['dup_num_2'][row[0]]) / np.array(user_accu_dict['count'][row[0]])
        
        # update dup_time matrix：時間を更新する
        dup_time_matrix[user_master[row[0]], content_master[row[1]]] = np.log(row[3] + 1)
        

    df_dup = pd.DataFrame({'dup_time':dup_time, 'dup_ratio':dup_ratio})
    test_df = pd.concat([test_df.reset_index(drop=True), df_dup], axis=1)

    return test_df, dup_time_matrix, user_accu_dict

## 特徴量の更新関数
- test時点で判明している変数(countなど）は、この時点で更新

In [25]:
def update_test_df(test_df, user_accu_dict, content_dict, tag_matrix, user_master, tag_mat_master, 
                   content_same_lag_accuracy_dict, content_same_prioraccu_accuracy_dict):
    
    
    # 初期化
    user_answer_sum = np.zeros(len(test_df), dtype=np.int32)
    user_answer_part_sum = np.zeros(len(test_df), dtype=np.float32)
    user_answer_L_sum = np.zeros(len(test_df), dtype=np.float32)
    user_answer_R_sum = np.zeros(len(test_df), dtype=np.float32)
    user_score_sum = np.zeros(len(test_df), dtype=np.float32)
    count = np.zeros(len(test_df), dtype=np.int32)
    count_part = np.zeros(len(test_df), dtype=np.int32)
    count_L = np.zeros(len(test_df), dtype=np.int32)
    count_R = np.zeros(len(test_df), dtype=np.int32)
    before_lecture = np.zeros(len(test_df), dtype=np.int8)
    first_user_ratio = np.zeros(len(test_df), dtype=np.float32)
    content_cumsum_ratio = np.zeros(len(test_df), dtype=np.float32)
    prior_accu_continuous = np.zeros(len(test_df), dtype=np.float32)
    prior_accu_no_continuous = np.zeros(len(test_df), dtype=np.float32)
    prior_user_answered_correctly = np.zeros(len(test_df), dtype=np.float32)
    prior_user_score = np.zeros(len(test_df), dtype=np.float32)
    prior_L_accu = np.zeros(len(test_df), dtype=np.float32)
    prior_R_accu = np.zeros(len(test_df), dtype=np.float32)
    
    lag_time = np.zeros(len(test_df), dtype=np.float64)
    prior_start_end_time = np.zeros(len(test_df), dtype=np.float64)
    start_to_start = np.zeros(len(test_df), dtype=np.float64)
    lec_num = np.zeros(len(test_df), dtype=np.int16)
    
    prior_session_time = np.zeros(len(test_df), dtype=np.float64)
    session_time = np.zeros(len(test_df), dtype=np.float64)
    session_first_time = np.zeros(len(test_df), dtype=np.float64)
    session_count = np.zeros(len(test_df), dtype=np.int16)
    session_accu = np.zeros(len(test_df), dtype=np.int16)
    prior_session_accu = np.zeros(len(test_df), dtype=np.float64)
    
    lag_user_ave = np.zeros(len(test_df), dtype=np.float32)
    lag_user_p2_ave = np.zeros(len(test_df), dtype=np.float32)
    lag_user_p5_ave = np.zeros(len(test_df), dtype=np.float32)
    lag_c_p2 = np.zeros(len(test_df), dtype=np.int32)
    lag_c_p5 = np.zeros(len(test_df), dtype=np.int32)
    lag_task = np.zeros(len(test_df), dtype=np.int16)
    dup_row_num = np.zeros(len(test_df), dtype=np.int16)
    
    count_dif_50_ratio = np.zeros(len(test_df), dtype=np.float32)
    count_dif_75_ratio = np.zeros(len(test_df), dtype=np.float32)
    count_dif_1_ratio = np.zeros(len(test_df), dtype=np.float32)
    difficulty_50_ratio = np.zeros(len(test_df), dtype=np.float32)
    difficulty_75_ratio = np.zeros(len(test_df), dtype=np.float32)
    difficulty_1_ratio = np.zeros(len(test_df), dtype=np.float32)
    
    prior_L_user_score = np.zeros(len(test_df), dtype=np.float32)
    prior_R_user_score = np.zeros(len(test_df), dtype=np.float32)
    user_score_ratio = np.zeros(len(test_df), dtype=np.float32)
    
    prior_p2_accu = np.zeros(len(test_df), dtype=np.float32)
    prior_p5_accu = np.zeros(len(test_df), dtype=np.float32)
    
    # add bin features
    lag_u_bin_0_accu_mean = np.zeros(len(test_df), dtype=np.float32)
    lag_u_bin_1_accu_mean = np.zeros(len(test_df), dtype=np.float32)
    lag_u_bin_2_accu_mean = np.zeros(len(test_df), dtype=np.float32)
    lag_u_bin_3_accu_mean = np.zeros(len(test_df), dtype=np.float32)
    lag_u_bin_0_count_ratio = np.zeros(len(test_df), dtype=np.float32)
    lag_u_bin_1_count_ratio = np.zeros(len(test_df), dtype=np.float32)
    lag_u_bin_2_count_ratio = np.zeros(len(test_df), dtype=np.float32)
    lag_u_bin_3_count_ratio = np.zeros(len(test_df), dtype=np.float32)

    elp_u_bin_0_accu_mean = np.zeros(len(test_df), dtype=np.float32)
    elp_u_bin_1_accu_mean = np.zeros(len(test_df), dtype=np.float32)
    elp_u_bin_2_accu_mean = np.zeros(len(test_df), dtype=np.float32)
    elp_u_bin_3_accu_mean = np.zeros(len(test_df), dtype=np.float32)
    elp_u_bin_0_count_ratio = np.zeros(len(test_df), dtype=np.float32)
    elp_u_bin_1_count_ratio = np.zeros(len(test_df), dtype=np.float32)
    elp_u_bin_2_count_ratio = np.zeros(len(test_df), dtype=np.float32)
    elp_u_bin_3_count_ratio = np.zeros(len(test_df), dtype=np.float32)
    
    content_accuracy = np.zeros(len(test_df), dtype=np.float32)
    same_lag_user_content_accuracy = np.zeros(len(test_df), dtype=np.float32)
        
    
    for cnt, row in enumerate(test_df[['user_id', 'part', 'content_id', 'L_R', 'timestamp', 'content_type_id', 
                                       'prior_question_elapsed_time', 'tag',  'task_container_id', 'prob_difficuly', 
                                       'bundle_id', 'prior_question_had_explanation']].values):
                
        # 各種dictの更新
        user_accu_dict['answered_correctly_cumsum'].setdefault(row[0], 0)
        user_accu_dict['count'].setdefault(row[0], 0)
        user_accu_dict['first_user_ratio'].setdefault(row[0], 0) 
        user_accu_dict['L_accu_sum'].setdefault(row[0], 0)
        user_accu_dict['R_accu_sum'].setdefault(row[0], 0)
        user_accu_dict['L_count'].setdefault(row[0], 0)
        user_accu_dict['R_count'].setdefault(row[0], 0)
        user_accu_dict['user_score_cumsum'].setdefault(row[0], 0)
        user_accu_dict['before_lecture'].setdefault(row[0], 0)        
        user_accu_dict['count_part'].setdefault(tuple(row[0:2]), 0)
        user_accu_dict['content_cumsum'].setdefault(row[0], 0)
        user_accu_dict['last_timestamp'].setdefault(row[0], 0)
        user_accu_dict['lag_time'].setdefault(row[0], np.nan)
        user_accu_dict['last_end_to_start_time'].setdefault(row[0], np.nan)
                
        user_accu_dict['session_time'].setdefault(row[0], np.nan)
        user_accu_dict['session_first_time'].setdefault(row[0], 0)
        user_accu_dict['session_count'].setdefault(row[0], 0)
        user_accu_dict['session_accu'].setdefault(row[0], 0)
        user_accu_dict['prior_session_time'].setdefault(row[0], np.nan)
        
        user_accu_dict['lag_user'].setdefault(row[0], 0)
        user_accu_dict['lag_user_p2'].setdefault(row[0], 0)
        user_accu_dict['lag_user_p5'].setdefault(row[0], 0)
        
        user_accu_dict['lag_c'].setdefault(row[0], 0)
        user_accu_dict['lag_c_p2'].setdefault(row[0], 0)
        user_accu_dict['lag_c_p5'].setdefault(row[0], 0)
        
        user_accu_dict['last_task_container_id'].setdefault(row[0], 0)
        user_accu_dict['lag_task'].setdefault(row[0], 1)

        user_accu_dict['prior_user_answered_correctly'].setdefault(row[0], np.nan)
        user_accu_dict['prior_user_score'].setdefault(row[0], np.nan)
        user_accu_dict['prior_L_accu'].setdefault(row[0], np.nan)
        user_accu_dict['prior_R_accu'].setdefault(row[0], np.nan)
        user_accu_dict['prior_accu_continuous'].setdefault(row[0], np.nan)
        user_accu_dict['prior_accu_no_continuous'].setdefault(row[0], np.nan)
        user_accu_dict['prior_session_prob'].setdefault(row[0], np.nan)
        user_accu_dict['prior_L_user_score'].setdefault(row[0], 0)
        user_accu_dict['prior_R_user_score'].setdefault(row[0], 0)
        user_accu_dict['prior_p2_accu'].setdefault(row[0], 0)
        user_accu_dict['prior_p5_accu'].setdefault(row[0], 0)
        
        user_accu_dict['count_dif_50'].setdefault(row[0], 0)
        user_accu_dict['count_dif_75'].setdefault(row[0], 0)
        user_accu_dict['count_dif_1'].setdefault(row[0], 0)
        user_accu_dict['difficulty_50'].setdefault(row[0], 0)
        user_accu_dict['difficulty_75'].setdefault(row[0], 0)
        user_accu_dict['difficulty_1'].setdefault(row[0], 0)
        
        user_accu_dict['last_bundle_id'].setdefault(row[0], np.nan)
        user_accu_dict['last_dup_row_num'].setdefault(row[0], 0)
        
        user_accu_dict['L_score'].setdefault(row[0], 0)
        user_accu_dict['R_score'].setdefault(row[0], 0)
        
        user_accu_dict['p2_count'].setdefault(row[0], 0)
        user_accu_dict['p5_count'].setdefault(row[0], 0)
        
        user_accu_dict['p2_accu_sum'].setdefault(row[0], 0)
        user_accu_dict['p5_accu_sum'].setdefault(row[0], 0)
        
        user_accu_dict['difficulty_50_ratio'].setdefault(row[0], 0)
        user_accu_dict['difficulty_75_ratio'].setdefault(row[0], 0)
        user_accu_dict['difficulty_1_ratio'].setdefault(row[0], 0)
        
        user_accu_dict['lag_time_bin'].setdefault(row[0], np.nan)
        user_accu_dict['elp_time_bin'].setdefault(row[0], np.nan)
        user_accu_dict['prior_user_answered_correctly_bin'].setdefault(row[0], np.nan)
        user_accu_dict['prior_answer'].setdefault(row[0], 0)
        
        user_accu_dict['lag_u_bin_0_accu_sum'].setdefault(row[0], 0)
        user_accu_dict['lag_u_bin_1_accu_sum'].setdefault(row[0], 0)
        user_accu_dict['lag_u_bin_2_accu_sum'].setdefault(row[0], 0) 
        user_accu_dict['lag_u_bin_3_accu_sum'].setdefault(row[0], 0)
        user_accu_dict['lag_u_bin_0_count'].setdefault(row[0], 0)
        user_accu_dict['lag_u_bin_1_count'].setdefault(row[0], 0)
        user_accu_dict['lag_u_bin_2_count'].setdefault(row[0], 0)
        user_accu_dict['lag_u_bin_3_count'].setdefault(row[0], 0)
        user_accu_dict['lag_u_bin_0_accu_mean'].setdefault(row[0], np.nan)
        user_accu_dict['lag_u_bin_1_accu_mean'].setdefault(row[0], np.nan)
        user_accu_dict['lag_u_bin_2_accu_mean'].setdefault(row[0], np.nan) 
        user_accu_dict['lag_u_bin_3_accu_mean'].setdefault(row[0], np.nan)
        
        user_accu_dict['elp_u_bin_0_accu_sum'].setdefault(row[0], 0)
        user_accu_dict['elp_u_bin_1_accu_sum'].setdefault(row[0], 0)
        user_accu_dict['elp_u_bin_2_accu_sum'].setdefault(row[0], 0)
        user_accu_dict['elp_u_bin_3_accu_sum'].setdefault(row[0], 0)
        user_accu_dict['elp_u_bin_0_count'].setdefault(row[0], 0)
        user_accu_dict['elp_u_bin_1_count'].setdefault(row[0], 0)
        user_accu_dict['elp_u_bin_2_count'].setdefault(row[0], 0)
        user_accu_dict['elp_u_bin_3_count'].setdefault(row[0], 0)
        user_accu_dict['elp_u_bin_0_accu_mean'].setdefault(row[0], np.nan)
        user_accu_dict['elp_u_bin_1_accu_mean'].setdefault(row[0], np.nan)
        user_accu_dict['elp_u_bin_2_accu_mean'].setdefault(row[0], np.nan) 
        user_accu_dict['elp_u_bin_3_accu_mean'].setdefault(row[0], np.nan)
        
        
        # matrix更新用のuser/tag masterもここで更新する
        # 新しいユーザーが来た場合は行列の一番最後の行に入れることにするため、len(user_master)を割り付ける
        user_master.setdefault(row[0], len(user_master))
        tag_mat_master.setdefault(row[7], 999)
        
        
        #---------------------------------
        # update dict before add feature
        #---------------------------------
        
        # timestampが前回と違うとき（＝bundle_idが違う。part6などの同じtimestampで複数問題があるときでも正しく更新するための条件）
        if row[4] != user_accu_dict['last_timestamp'][row[0]]:
                                    
            # 'prior_question_elapsed_time'がnanでない場合、end_to_start_timeの辞書を更新
            if row[6] >= 0:
                user_accu_dict['last_end_to_start_time'][row[0]] = np.array(user_accu_dict['lag_time'][row[0]]) - np.array(row[6])
                
            # lag_timeを更新。前回時間からの差分
            user_accu_dict['lag_time'][row[0]] = np.array(row[4]) - np.array(user_accu_dict['last_timestamp'][row[0]])
                        
            # task_containerの差分を更新
            user_accu_dict['lag_task'][row[0]] = row[8] - user_accu_dict['last_task_container_id'][row[0]]
            user_accu_dict['last_task_container_id'][row[0]] = row[8]
            
            # timestampが異なるから本来はbundle_idが違うはずなのに、連続して同じbundle_idが来る場合はカウント（同じ問題の連続回答フラグ）
            if row[5] == 0:
                if row[10] == user_accu_dict['last_bundle_id'][row[0]]:
                    user_accu_dict['last_dup_row_num'][row[0]] += 1
                else:
                    user_accu_dict['last_dup_row_num'][row[0]] = 0
            # 辞書更新
            user_accu_dict['last_bundle_id'][row[0]] = row[10]

        
        #---------------------------------
        # update tag_num matrix
        #---------------------------------
        
        # もしlectureだった場合、tagと合わせて回数を更新
        if row[5] == 1:
            tag_matrix[user_master[row[0]], tag_mat_master[row[7]]] += 1
                

        #---------------------------------
        # update without lecture feature
        #---------------------------------
        
        if row[5] == 0:

            # update count dict
            user_accu_dict['count'][row[0]] += 1
            user_accu_dict['count_part'][tuple(row[0:2])] += 1
            
            # update content_cumsum
            #'answered_correctly_content_first_look'はコンテンツの難易度。それをどんどん合計していく（後からcountで割る）
            user_accu_dict['content_cumsum'][row[0]] += content_dict['answered_correctly_content_first_look'][row[2]]
            
            # LもしくはRならcountup
            if row[3] == 1:
                user_accu_dict['L_count'][row[0]] += 1
            else:
                user_accu_dict['R_count'][row[0]] += 1
            
            # part2/5ならcountup
            if row[1] == 2:
                user_accu_dict['p2_count'][row[0]] += 1
 
            if row[1] == 5:
                user_accu_dict['p5_count'][row[0]] += 1
                       
            # first_user_ratio
            # timestampが0のとき(初めてのとき）のみ、初回質問の回答率を付与する（以後ユーザー内での更新無）
            if row[4] == 0:
                user_accu_dict['first_user_ratio'][row[0]] = content_dict['content_user_cover_ratio'][row[2]]
                
                
            #---------------------------------
            # update session feature
            #---------------------------------
            # lag_timeが小さい(30分以内）場合、session_countを更新する
            # lag_timeが大きい場合、session_time(session_first_time - last_timestamp)を求める 
            # また、session_count/accuを0に更新してから（リスタート）session_starttimeをtimestampに更新する

            if user_accu_dict['lag_time'][row[0]] < 3600000 or row[4] == 0:
                user_accu_dict['session_count'][row[0]] += 1
            else:
                if row[4] != user_accu_dict['last_timestamp'][row[0]]:
                    user_accu_dict['session_time'][row[0]] = user_accu_dict['last_timestamp'][row[0]] - user_accu_dict['session_first_time'][row[0]]                        
                # update count
                user_accu_dict['session_count'][row[0]] = 1
                user_accu_dict['session_first_time'][row[0]] = row[4]
                
            #---------------------------------
            # update lagtime feature(30分以上はカウント無。part毎でも求める）
            #---------------------------------
            
            if user_accu_dict['lag_time'][row[0]] < 900000:
                              
                user_accu_dict['lag_user'][row[0]] += user_accu_dict['lag_time'][row[0]]
                user_accu_dict['lag_c'][row[0]] += 1
            
                if row[1] == 2:
                    user_accu_dict['lag_user_p2'][row[0]] += user_accu_dict['lag_time'][row[0]]
                    user_accu_dict['lag_c_p2'][row[0]] += 1

                if row[1] == 5:
                    user_accu_dict['lag_user_p5'][row[0]] += user_accu_dict['lag_time'][row[0]]
                    user_accu_dict['lag_c_p5'][row[0]] += 1
            
            
            #---------------------------------
            # update difficulty feature
            #---------------------------------
            
            # 各難易度に合わせて難易度count更新
            if row[9] == '0.5':
                user_accu_dict['count_dif_50'][row[0]] += 1
                
            if row[9] == '0.75':
                user_accu_dict['count_dif_75'][row[0]] += 1
                
            if row[9] == '1.0':
                user_accu_dict['count_dif_1'][row[0]] += 1
                
                
            #---------------------------------
            # make bin features
            #---------------------------------
            # 紐づけ用のbin featureを作成する
            user_accu_dict['prior_user_answered_correctly_bin'][row[0]] = bin_accuracy(user_accu_dict['prior_user_answered_correctly'][row[0]])
            user_accu_dict['lag_time_bin'][row[0]] = bin_lag_time(user_accu_dict['lag_time'][row[0]])
            user_accu_dict['elp_time_bin'][row[0]] = bin_elp_time(row[6])
            
            # dictを更新 (binが無いと作れないのでここで更新)
            content_same_lag_accuracy_dict['same_lag_user_content_accuracy'].setdefault(tuple([row[2], user_accu_dict['lag_time_bin'][row[0]]]), np.nan)
            content_same_prioraccu_accuracy_dict['same_accu_user_content_accuracy'].setdefault(tuple([row[2], user_accu_dict['prior_user_answered_correctly_bin'][row[0]]]), np.nan)
            
            
            #---------------------------------
            # make user_bin features
            #---------------------------------
            
            if user_accu_dict['lag_time_bin'][row[0]] == 0:
                user_accu_dict['lag_u_bin_0_count'][row[0]] += 1
                
            if user_accu_dict['lag_time_bin'][row[0]] == 1:
                user_accu_dict['lag_u_bin_1_count'][row[0]] += 1
                
            if user_accu_dict['lag_time_bin'][row[0]] == 2:
                user_accu_dict['lag_u_bin_2_count'][row[0]] += 1
                
            if user_accu_dict['lag_time_bin'][row[0]] == 3:
                user_accu_dict['lag_u_bin_3_count'][row[0]] += 1
                
                
            if user_accu_dict['elp_time_bin'][row[0]] == 0:
                user_accu_dict['elp_u_bin_0_count'][row[0]] += 1
                
            if user_accu_dict['elp_time_bin'][row[0]] == 1:
                user_accu_dict['elp_u_bin_1_count'][row[0]] += 1
                
            if user_accu_dict['elp_time_bin'][row[0]] == 2:
                user_accu_dict['elp_u_bin_2_count'][row[0]] += 1
                
            if user_accu_dict['elp_time_bin'][row[0]] == 3:
                user_accu_dict['elp_u_bin_3_count'][row[0]] += 1
            
            # content_accuracyの入力(content_idとbinで紐づける)
            # accuracyはtimestamp0のときにnullにする
            #if user_accu_dict['prior_user_answered_correctly_bin'][row[0]] >= 0:
            
            if row[4] > 0:
                content_accuracy[cnt] = content_same_prioraccu_accuracy_dict['same_accu_user_content_accuracy'][
                    tuple([row[2], user_accu_dict['prior_user_answered_correctly_bin'][row[0]]])]
            else:
                content_accuracy[cnt] = np.nan
                
            same_lag_user_content_accuracy[cnt] = content_same_lag_accuracy_dict['same_lag_user_content_accuracy'][
                tuple([row[2], user_accu_dict['lag_time_bin'][row[0]]])]
                

        #---------------------------------
        # add feature
        #---------------------------------
        
        user_answer_sum[cnt] =     user_accu_dict['answered_correctly_cumsum'][row[0]]
        count[cnt] =               user_accu_dict['count'][row[0]]
        count_part[cnt] =          user_accu_dict['count_part'][tuple(row[0:2])]
        count_L[cnt] =             user_accu_dict['L_count'][row[0]]
        count_R[cnt] =             user_accu_dict['R_count'][row[0]]
        before_lecture[cnt] =      user_accu_dict['before_lecture'][row[0]]
        first_user_ratio[cnt] =    user_accu_dict['first_user_ratio'][row[0]]
        content_cumsum_ratio[cnt] = np.array(user_accu_dict['content_cumsum'][row[0]]) / np.array(user_accu_dict['count'][row[0]])
        prior_accu_continuous[cnt] = user_accu_dict['prior_accu_continuous'][row[0]]
        prior_accu_no_continuous[cnt] = user_accu_dict['prior_accu_no_continuous'][row[0]]
        
        lag_time[cnt] = user_accu_dict['lag_time'][row[0]]
        prior_start_end_time[cnt] = user_accu_dict['last_end_to_start_time'][row[0]]
        
        lec_num[cnt] = tag_matrix[user_master[row[0]], tag_mat_master[row[7]]]
        session_time[cnt] = row[4] - user_accu_dict['session_first_time'][row[0]]
        prior_session_time[cnt] = user_accu_dict['session_time'][row[0]]
        session_count[cnt] = user_accu_dict['session_count'][row[0]]
        lag_user_ave[cnt] = np.array(user_accu_dict['lag_user'][row[0]]) / np.array(user_accu_dict['lag_c'][row[0]])
        lag_user_p2_ave[cnt] = np.array(user_accu_dict['lag_user_p2'][row[0]]) / np.array(user_accu_dict['lag_c_p2'][row[0]])
        lag_user_p5_ave[cnt] = np.array(user_accu_dict['lag_user_p5'][row[0]]) / np.array(user_accu_dict['lag_c_p5'][row[0]])
        lag_c_p2[cnt] = user_accu_dict['lag_c_p2'][row[0]]
        lag_c_p5[cnt] = user_accu_dict['lag_c_p5'][row[0]]
        
        lag_task[cnt] = user_accu_dict['lag_task'][row[0]]        
        dup_row_num[cnt] = user_accu_dict['last_dup_row_num'][row[0]]
        
        count_dif_50_ratio[cnt] = np.array(user_accu_dict['count_dif_50'][row[0]]) / np.array(user_accu_dict['count'][row[0]])
        count_dif_75_ratio[cnt] = np.array(user_accu_dict['count_dif_75'][row[0]]) / np.array(user_accu_dict['count'][row[0]])
        count_dif_1_ratio[cnt] = np.array(user_accu_dict['count_dif_1'][row[0]]) / np.array(user_accu_dict['count'][row[0]])

        
        # lag_time_bin_features
        lag_u_bin_0_accu_mean[cnt] = user_accu_dict['lag_u_bin_0_accu_mean'][row[0]]
        lag_u_bin_1_accu_mean[cnt] = user_accu_dict['lag_u_bin_1_accu_mean'][row[0]]
        lag_u_bin_2_accu_mean[cnt] = user_accu_dict['lag_u_bin_2_accu_mean'][row[0]]
        lag_u_bin_3_accu_mean[cnt] = user_accu_dict['lag_u_bin_3_accu_mean'][row[0]]
        
        elp_u_bin_0_accu_mean[cnt] = user_accu_dict['elp_u_bin_0_accu_mean'][row[0]]
        elp_u_bin_1_accu_mean[cnt] = user_accu_dict['elp_u_bin_1_accu_mean'][row[0]]
        elp_u_bin_2_accu_mean[cnt] = user_accu_dict['elp_u_bin_2_accu_mean'][row[0]]
        elp_u_bin_3_accu_mean[cnt] = user_accu_dict['elp_u_bin_3_accu_mean'][row[0]]
        
        lag_u_bin_0_count_ratio[cnt] = np.array(user_accu_dict['lag_u_bin_0_count'][row[0]]) / np.array(user_accu_dict['count'][row[0]])
        lag_u_bin_1_count_ratio[cnt] = np.array(user_accu_dict['lag_u_bin_1_count'][row[0]]) / np.array(user_accu_dict['count'][row[0]])
        lag_u_bin_2_count_ratio[cnt] = np.array(user_accu_dict['lag_u_bin_2_count'][row[0]]) / np.array(user_accu_dict['count'][row[0]])
        lag_u_bin_3_count_ratio[cnt] = np.array(user_accu_dict['lag_u_bin_3_count'][row[0]]) / np.array(user_accu_dict['count'][row[0]])
        
        elp_u_bin_0_count_ratio[cnt] = np.array(user_accu_dict['elp_u_bin_0_count'][row[0]]) / np.array(user_accu_dict['count'][row[0]])
        elp_u_bin_1_count_ratio[cnt] = np.array(user_accu_dict['elp_u_bin_1_count'][row[0]]) / np.array(user_accu_dict['count'][row[0]])
        elp_u_bin_2_count_ratio[cnt] = np.array(user_accu_dict['elp_u_bin_2_count'][row[0]]) / np.array(user_accu_dict['count'][row[0]])
        elp_u_bin_3_count_ratio[cnt] = np.array(user_accu_dict['elp_u_bin_3_count'][row[0]]) / np.array(user_accu_dict['count'][row[0]])
        
          
                  
        # timestamp=0ならnanにするものたち
        if row[4] != 0:
            
            prior_session_accu[cnt] = user_accu_dict['prior_session_prob'][row[0]]
            
            prior_user_answered_correctly[cnt] = user_accu_dict['prior_user_answered_correctly'][row[0]]
            prior_user_score[cnt] = user_accu_dict['prior_user_score'][row[0]]
            prior_L_accu[cnt] = user_accu_dict['prior_L_accu'][row[0]]
            prior_R_accu[cnt] = user_accu_dict['prior_R_accu'][row[0]]
            
            prior_L_user_score[cnt] = user_accu_dict['prior_L_user_score'][row[0]]
            prior_R_user_score[cnt] = user_accu_dict['prior_R_user_score'][row[0]]
            
            difficulty_50_ratio[cnt] = user_accu_dict['difficulty_50_ratio'][row[0]]
            difficulty_75_ratio[cnt] = user_accu_dict['difficulty_75_ratio'][row[0]]
            difficulty_1_ratio[cnt] = user_accu_dict['difficulty_1_ratio'][row[0]]
            
            prior_p2_accu[cnt] = user_accu_dict['prior_p2_accu'][row[0]]
            prior_p5_accu[cnt] = user_accu_dict['prior_p5_accu'][row[0]]
            
            lag_u_bin_0_accu_mean[cnt] = user_accu_dict['lag_u_bin_0_accu_mean'][row[0]]
            lag_u_bin_1_accu_mean[cnt] = user_accu_dict['lag_u_bin_1_accu_mean'][row[0]]
            lag_u_bin_2_accu_mean[cnt] = user_accu_dict['lag_u_bin_2_accu_mean'][row[0]]
            lag_u_bin_3_accu_mean[cnt] = user_accu_dict['lag_u_bin_3_accu_mean'][row[0]]

            elp_u_bin_0_accu_mean[cnt] = user_accu_dict['elp_u_bin_0_accu_mean'][row[0]]
            elp_u_bin_1_accu_mean[cnt] = user_accu_dict['elp_u_bin_1_accu_mean'][row[0]]
            elp_u_bin_2_accu_mean[cnt] = user_accu_dict['elp_u_bin_2_accu_mean'][row[0]]
            elp_u_bin_3_accu_mean[cnt] = user_accu_dict['elp_u_bin_3_accu_mean'][row[0]]
            
        else:
            prior_L_user_score[cnt] = np.nan
            prior_R_user_score[cnt] = np.nan
            
            difficulty_50_ratio[cnt] = np.nan
            difficulty_75_ratio[cnt] = np.nan
            difficulty_1_ratio[cnt] = np.nan
            
            prior_session_accu[cnt] = np.nan
            
            prior_user_answered_correctly[cnt] = np.nan
            prior_user_score[cnt] = np.nan
            prior_L_accu[cnt] = np.nan
            prior_R_accu[cnt] = np.nan
            prior_p2_accu[cnt] = np.nan
            prior_p5_accu[cnt] = np.nan
            
            lag_u_bin_0_accu_mean[cnt] = np.nan
            lag_u_bin_1_accu_mean[cnt] = np.nan
            lag_u_bin_2_accu_mean[cnt] = np.nan
            lag_u_bin_3_accu_mean[cnt] = np.nan

            elp_u_bin_0_accu_mean[cnt] = np.nan
            elp_u_bin_1_accu_mean[cnt] = np.nan
            elp_u_bin_2_accu_mean[cnt] = np.nan
            elp_u_bin_3_accu_mean[cnt] = np.nan
        
        
        #---------------------------------
        # update dict after add feature
        #---------------------------------
        
        # update before_lecture
        # timestampが変わらない場合はbefore_lectureは更新しない
        if row[4] != user_accu_dict['last_timestamp'][row[0]]:
            user_accu_dict['before_lecture'][row[0]] = 0
                
        # update last timestamp
        user_accu_dict['last_timestamp'][row[0]] = row[4]
        
    
    user_score_df = pd.DataFrame({'prior_user_answered_correctly': prior_user_answered_correctly,
                                  'prior_user_score': prior_user_score,
                                  'prior_L_accu': prior_L_accu,
                                  'prior_R_accu': prior_R_accu,
                                  'count': count,
                                  'count_part': count_part,
                                  'L_count': count_L,
                                  'R_count': count_R,
                                  'before_lecture': before_lecture,
                                  'lag_time':lag_time,
                                  'first_user_ratio':first_user_ratio,
                                  'prior_accu_continuous':prior_accu_continuous,
                                  'prior_accu_no_continuous':prior_accu_no_continuous,
                                  'content_cumsum_ratio':content_cumsum_ratio,
                                  'prior_end_to_start':prior_start_end_time,
                                  'start_to_start':start_to_start,
                                  
                                  'lec_num':lec_num,
                                  
                                  'session_time':session_time,
                                  'prior_session_time':prior_session_time,
                                  'session_count':session_count,
                                  'prior_session_prob':prior_session_accu,
                                  'lag_user_ave':lag_user_ave,
                                  'lag_user_p2_ave':lag_user_p2_ave,
                                  'lag_user_p5_ave':lag_user_p5_ave,
                                  'lag_c_p2':lag_c_p2,
                                  'lag_c_p5':lag_c_p5,
                                  
                                  'count_dif_50_ratio':count_dif_50_ratio,
                                  'count_dif_75_ratio':count_dif_75_ratio,
                                  'count_dif_1_ratio':count_dif_1_ratio,
                                  'difficulty_50_ratio':difficulty_50_ratio,
                                  'difficulty_75_ratio':difficulty_75_ratio,
                                  'difficulty_1_ratio':difficulty_1_ratio,
                                  
                                  'prior_L_user_score':prior_L_user_score,
                                  'prior_R_user_score':prior_R_user_score,
                                  
                                  'lag_task':lag_task,
                                  'dup_row_num':dup_row_num,
                                  'prior_p2_accu':prior_p2_accu,
                                  'prior_p5_accu':prior_p5_accu,
                                  
                                  'lag_u_bin_0_accu_mean':lag_u_bin_0_accu_mean,
                                  'lag_u_bin_1_accu_mean':lag_u_bin_1_accu_mean,
                                  'lag_u_bin_2_accu_mean':lag_u_bin_2_accu_mean,
                                  'lag_u_bin_3_accu_mean':lag_u_bin_3_accu_mean,
                                  
                                  'lag_u_bin_0_count_ratio':lag_u_bin_0_count_ratio,
                                  'lag_u_bin_1_count_ratio':lag_u_bin_1_count_ratio,
                                  'lag_u_bin_2_count_ratio':lag_u_bin_2_count_ratio,
                                  'lag_u_bin_3_count_ratio':lag_u_bin_3_count_ratio,
                                  
                                  'elp_u_bin_0_accu_mean':elp_u_bin_0_accu_mean,
                                  'elp_u_bin_1_accu_mean':elp_u_bin_1_accu_mean,
                                  'elp_u_bin_2_accu_mean':elp_u_bin_2_accu_mean,
                                  'elp_u_bin_3_accu_mean':elp_u_bin_3_accu_mean,
                                  
                                  'elp_u_bin_0_count_ratio':elp_u_bin_0_count_ratio,
                                  'elp_u_bin_1_count_ratio':elp_u_bin_1_count_ratio,
                                  'elp_u_bin_2_count_ratio':elp_u_bin_2_count_ratio,
                                  'elp_u_bin_3_count_ratio':elp_u_bin_3_count_ratio,
                                  
                                  'content_accuracy':content_accuracy,
                                  'same_lag_user_content_accuracy':same_lag_user_content_accuracy,
                              })
    
   
    # add user_score_ratio
    user_score_df['user_score_ratio'] = np.array(user_score_df['prior_L_user_score']) / np.array(user_score_df['prior_R_user_score'])
    
    test_df = pd.concat([test_df.reset_index(drop=True), 
                         user_score_df[['count',
                                     'count_part',
                                     'L_count',
                                     'R_count',
                                     'prior_user_answered_correctly',
                                     'prior_user_score',
                                     'prior_L_accu',
                                     'prior_R_accu',
                                     'before_lecture',
                                     'lag_time',
                                     'first_user_ratio',
                                     'content_cumsum_ratio',
                                     'prior_accu_continuous',
                                     'prior_accu_no_continuous',
                                     'prior_end_to_start',
                                     'lec_num',
                                     'session_time',
                                     'prior_session_time',
                                     'session_count',
                                     'prior_session_prob',
                                     'lag_user_ave',
                                     'lag_user_p2_ave',
                                     'lag_user_p5_ave',
                                     'lag_c_p2',
                                     'lag_c_p5',
                                     'lag_task',
                                     'dup_row_num',
                                     'count_dif_50_ratio', 
                                     'count_dif_75_ratio', 
                                     'count_dif_1_ratio',
                                     'difficulty_50_ratio',
                                     'difficulty_75_ratio',
                                     'difficulty_1_ratio',
                                     'prior_L_user_score',
                                     'prior_R_user_score',
                                     'user_score_ratio',
                                     'prior_p2_accu',
                                     'prior_p5_accu',
                                     
                                     # lag_time_bin_features
                                     'lag_u_bin_0_accu_mean',
                                     'lag_u_bin_1_accu_mean',
                                     'lag_u_bin_2_accu_mean',
                                     'lag_u_bin_3_accu_mean',
                                        
                                     'lag_u_bin_0_count_ratio',
                                     'lag_u_bin_1_count_ratio',
                                     'lag_u_bin_2_count_ratio',
                                     'lag_u_bin_3_count_ratio',
                                        
                                     'elp_u_bin_0_accu_mean',
                                     'elp_u_bin_1_accu_mean',
                                     'elp_u_bin_2_accu_mean',
                                     'elp_u_bin_3_accu_mean',
                                        
                                     'elp_u_bin_0_count_ratio',
                                     'elp_u_bin_1_count_ratio',
                                     'elp_u_bin_2_count_ratio',
                                     'elp_u_bin_3_count_ratio',
                                        
                                     'content_accuracy',
                                     'same_lag_user_content_accuracy',
                                                                                
                                    ]]], axis=1)

    return test_df, user_accu_dict, tag_matrix, user_master, tag_mat_master, content_same_prioraccu_accuracy_dict, content_same_lag_accuracy_dict

## 辞書の更新関数（前問の回答をもとに更新する辞書）

In [26]:
def update_dict(test_df, user_accu_dict, answer_score_dict):
    
    for cnt, row in enumerate(test_df[['user_id', 'part', 'content_id', 'user_answer', 
                                       'answered_correctly', 'content_type_id', 'L_R', 'timestamp', 'prob_difficuly']].values):
                
        
        if row[5] == 0:
            
            # answer_score_dictは新しいcontent*選択肢の組み合わせが入る可能性があるのでここで更新
            answer_score_dict['answer_score'].setdefault(tuple(row[2:4]), 0.5)
            
            
            # answer_scoreを取得
            get_answer_score = answer_score_dict['answer_score'][tuple(row[2:4])]
            
            
            #----------------------------------
            # update dict
            #----------------------------------
            
            # ユーザー正答率   
            user_accu_dict['answered_correctly_cumsum'][row[0]] += row[4]          
            
            # L/R正解追加
            if row[6] == 1:
                user_accu_dict['L_accu_sum'][row[0]] += row[4]
                user_accu_dict['L_score'][row[0]] += get_answer_score
            else:
                user_accu_dict['R_accu_sum'][row[0]] += row[4]
                user_accu_dict['R_score'][row[0]] += get_answer_score
            
            # 連続正解:正しい場合は+1を追加し続け, 間違っていたら0に戻す
            if row[4] == 1:
                user_accu_dict['prior_accu_continuous'][row[0]] += 1
                user_accu_dict['prior_accu_no_continuous'][row[0]] = 0
            else:
                user_accu_dict['prior_accu_continuous'][row[0]] = 0
                user_accu_dict['prior_accu_no_continuous'][row[0]] += 1
            
            # user_scoreの追加
            user_accu_dict['user_score_cumsum'][row[0]] += get_answer_score
            
            # session（30分以上のlag_timeがない）内の回答ならカウントアップ
            if user_accu_dict['lag_time'][row[0]] < 3600000 or row[7] == 0:
                user_accu_dict['session_accu'][row[0]] += row[4]
            else:
                # もしlag_timeの差分が大きい場合は0に更新してからカウントアップ
                user_accu_dict['session_accu'][row[0]] = 0
                user_accu_dict['session_accu'][row[0]] += row[4]
            
            # part2/5
            if row[1] == 2:
                user_accu_dict['p2_accu_sum'][row[0]] += row[4]
                
            if row[1] == 5:
                user_accu_dict['p5_accu_sum'][row[0]] += row[4]
            
            # difficulty
            if row[8] == '0.5':
                user_accu_dict['difficulty_50'][row[0]] += row[4]
                
            if row[8] == '0.75':
                user_accu_dict['difficulty_75'][row[0]] += row[4]
                
            if row[8] == '1.0':
                user_accu_dict['difficulty_1'][row[0]] += row[4]
                
            #---------------------------------
            # make user_bin accu features
            #---------------------------------
            
            if user_accu_dict['lag_time_bin'][row[0]] == 0:
                user_accu_dict['lag_u_bin_0_accu_sum'][row[0]] += row[4]
                
            if user_accu_dict['lag_time_bin'][row[0]] == 1:
                user_accu_dict['lag_u_bin_1_accu_sum'][row[0]] += row[4]
                
            if user_accu_dict['lag_time_bin'][row[0]] == 2:
                user_accu_dict['lag_u_bin_2_accu_sum'][row[0]] += row[4]
                
            if user_accu_dict['lag_time_bin'][row[0]] == 3:
                user_accu_dict['lag_u_bin_3_accu_sum'][row[0]] += row[4]
                
            # elapse_timeは前問の時間のため、前問の正解を紐づける
            if user_accu_dict['elp_time_bin'][row[0]] == 0:
                user_accu_dict['elp_u_bin_0_accu_sum'][row[0]] += user_accu_dict['prior_answer'][row[0]]
                
            if user_accu_dict['elp_time_bin'][row[0]] == 1:
                user_accu_dict['elp_u_bin_1_accu_sum'][row[0]] += user_accu_dict['prior_answer'][row[0]]
                
            if user_accu_dict['elp_time_bin'][row[0]] == 2:
                user_accu_dict['elp_u_bin_2_accu_sum'][row[0]] += user_accu_dict['prior_answer'][row[0]]
                
            if user_accu_dict['elp_time_bin'][row[0]] == 3:
                user_accu_dict['elp_u_bin_3_accu_sum'][row[0]] += user_accu_dict['prior_answer'][row[0]]
                
            # 全問の正解を保存
            user_accu_dict['prior_answer'][row[0]] = row[4]
                            
                
        else:
            # before_lesson
            user_accu_dict['before_lecture'][row[0]] = 1
            
        #----------------------------------
        # update prior prob features
        #----------------------------------
        # update prior features
        # prob系の変数は、前回回答の答えが得られた後に更新する
        # countは前問時点のcountが入るので、回答数とcountは揃う
        user_accu_dict['prior_user_answered_correctly'][row[0]] = np.array(user_accu_dict['answered_correctly_cumsum'][row[0]]) / np.array(user_accu_dict['count'][row[0]])
        user_accu_dict['prior_user_score'][row[0]] = np.array(user_accu_dict['user_score_cumsum'][row[0]]) / np.array(user_accu_dict['count'][row[0]])
        user_accu_dict['prior_L_accu'][row[0]] = np.array(user_accu_dict['L_accu_sum'][row[0]]) / np.array( user_accu_dict['L_count'][row[0]])
        user_accu_dict['prior_R_accu'][row[0]] = np.array(user_accu_dict['R_accu_sum'][row[0]]) / np.array( user_accu_dict['R_count'][row[0]])
        user_accu_dict['prior_session_prob'][row[0]] = np.array(user_accu_dict['session_accu'][row[0]]) / np.array(user_accu_dict['session_count'][row[0]])
        user_accu_dict['prior_L_user_score'][row[0]] = np.array(user_accu_dict['L_score'][row[0]]) / np.array(user_accu_dict['L_count'][row[0]])
        user_accu_dict['prior_R_user_score'][row[0]] = np.array(user_accu_dict['R_score'][row[0]]) / np.array(user_accu_dict['R_count'][row[0]])
        user_accu_dict['difficulty_50_ratio'][row[0]] = np.array(user_accu_dict['difficulty_50'][row[0]]) / np.array(user_accu_dict['count_dif_50'][row[0]])
        user_accu_dict['difficulty_75_ratio'][row[0]] = np.array(user_accu_dict['difficulty_75'][row[0]]) / np.array(user_accu_dict['count_dif_75'][row[0]])
        user_accu_dict['difficulty_1_ratio'][row[0]] = np.array(user_accu_dict['difficulty_1'][row[0]]) / np.array(user_accu_dict['count_dif_1'][row[0]])
        user_accu_dict['prior_p2_accu'][row[0]] = np.array(user_accu_dict['p2_accu_sum'][row[0]]) / np.array(user_accu_dict['p2_count'][row[0]])
        user_accu_dict['prior_p5_accu'][row[0]] = np.array(user_accu_dict['p5_accu_sum'][row[0]]) / np.array(user_accu_dict['p5_count'][row[0]])
        
        # lag_time_bin
        user_accu_dict['lag_u_bin_0_accu_mean'][row[0]] = np.array(user_accu_dict['lag_u_bin_0_accu_sum'][row[0]]) / np.array(user_accu_dict['lag_u_bin_0_count'][row[0]])
        user_accu_dict['lag_u_bin_1_accu_mean'][row[0]] = np.array(user_accu_dict['lag_u_bin_1_accu_sum'][row[0]]) / np.array(user_accu_dict['lag_u_bin_1_count'][row[0]])
        user_accu_dict['lag_u_bin_2_accu_mean'][row[0]] = np.array(user_accu_dict['lag_u_bin_2_accu_sum'][row[0]]) / np.array(user_accu_dict['lag_u_bin_2_count'][row[0]])
        user_accu_dict['lag_u_bin_3_accu_mean'][row[0]] = np.array(user_accu_dict['lag_u_bin_3_accu_sum'][row[0]]) / np.array(user_accu_dict['lag_u_bin_3_count'][row[0]])
        
        user_accu_dict['elp_u_bin_0_accu_mean'][row[0]] = np.array(user_accu_dict['elp_u_bin_0_accu_sum'][row[0]]) / np.array(user_accu_dict['elp_u_bin_0_count'][row[0]])
        user_accu_dict['elp_u_bin_1_accu_mean'][row[0]] = np.array(user_accu_dict['elp_u_bin_1_accu_sum'][row[0]]) / np.array(user_accu_dict['elp_u_bin_1_count'][row[0]])
        user_accu_dict['elp_u_bin_2_accu_mean'][row[0]] = np.array(user_accu_dict['elp_u_bin_2_accu_sum'][row[0]]) / np.array(user_accu_dict['elp_u_bin_2_count'][row[0]])
        user_accu_dict['elp_u_bin_3_accu_mean'][row[0]] = np.array(user_accu_dict['elp_u_bin_3_accu_sum'][row[0]]) / np.array(user_accu_dict['elp_u_bin_3_count'][row[0]])
        
        

## validationかどうかの確認
- validationで確認したい場合はvalidation_flgをTrueにする
- 本番で回すときはFalseにする

In [27]:
train_header = pd.read_csv('../input/riiid-test-answer-prediction/train.csv', nrows = 1)

if use_train:
    # simulation data
    #train_samp = pd.read_csv('../input/riiid-test-answer-prediction/train.csv', skiprows=77000, nrows = 2000)
    train_samp = pd.read_csv('../input/riiid-test-answer-prediction/train.csv', skiprows=100000, nrows = 100)
    #train_samp = pd.read_csv('../input/riiid-test-answer-prediction/train.csv', nrows = 1000)
    train_samp.columns = train_header.columns
    iter_test = Iter_Valid(train_samp,max_user=1)
    predicted = []
    def set_predict(df):
        predicted.append(df)
else:
    import riiideducation
    env = riiideducation.make_env()
    iter_test = env.iter_test()
    set_predict = env.predict

In [28]:
%%time

# set df
previous_test_df = None

# set time
start = time.time()

print('start batch')


contents_use_col = ['answered_correctly_content',
                    'content_user_cover_ratio',
                    'answered_correctly_content_first_look',
                    'fix_answered_correctly_content_first_look',
                    'content_accu_mean',
                    'ave_elapsed_time',
                    'content_series_mean',
                    'mean_lag_time',
                    
                     # lag
                     'content_lag_count_dist_0',
                     'content_lag_count_dist_1',
                     'content_lag_count_dist_2',
                     'content_lag_count_dist_3',
                     'same_lagbin_content_acc_mean_0',
                     'same_lagbin_content_acc_mean_1',
                     'same_lagbin_content_acc_mean_2',
                     'same_lagbin_content_acc_mean_3',

                     # elp
                     'content_elp_count_dist_0',
                     'content_elp_count_dist_1',
                     'content_elp_count_dist_2',
                     'content_elp_count_dist_3',
                     'same_elpbin_content_acc_mean_0',
                     'same_elpbin_content_acc_mean_1',
                     'same_elpbin_content_acc_mean_2',
                     'same_elpbin_content_acc_mean_3',

                     # acc
                     'content_bin_count_ratio_0',
                     'content_bin_count_ratio_1',
                     'content_bin_count_ratio_2',
                     'content_bin_count_ratio_3',
                     'same_prioracbin_content_acc_mean_0',
                     'same_prioracbin_content_acc_mean_1',
                     'same_prioracbin_content_acc_mean_2',
                     'same_prioracbin_content_acc_mean_3',
                    
                     # ninopy content features
                     'content_unacc_user_prior_user_answered_correctly_mean',
                     'content_unacc_user_prior_user_answered_correctly_std',
                    
                     # additinal features
                     'content_dup_ratio', 
                     'content_dup_count_ratio',
                     'bin_0_accu',
                     'bin_1_accu',
                     'bin_2_accu',
                     'bin_3_accu',
                     'bin_4_accu',
                    
                   ] + ['vec_w2v_' + str(i) for i in range(0, 10)]

# debug用(sub前に推論DMの中身を確認する時に利用)
all_test = pd.DataFrame()


for (test_df, sample_prediction_df) in iter_test:
    
    if previous_test_df is not None:
        
        # Get prior features
        previous_test_df['answered_correctly'] = eval(test_df['prior_group_answers_correct'].iloc[0])
        previous_test_df['user_answer'] = eval(test_df['prior_group_responses'].iloc[0])
        
        
        # update dict
        update_dict(previous_test_df, user_accu_dict, answer_score_dict)
        
        test_df = pd.concat([test_df.reset_index(drop=True), 
                             questions.reindex(test_df['content_id'].values).reset_index(drop=True)], axis=1)
        previous_test_df = test_df.copy()
        
    else: 
        test_df = pd.concat([test_df.reset_index(drop=True), 
                             questions.reindex(test_df['content_id'].values).reset_index(drop=True)], axis=1)
        
        previous_test_df = test_df.copy() 
        
    # make features
    test_df, user_accu_dict, tag_matrix, user_master, tag_mat_master, \
    content_same_prioraccu_accuracy_dict, content_same_lag_accuracy_dict = update_test_df(
        test_df, user_accu_dict, content_dict, tag_matrix, user_master, tag_mat_master, 
        content_same_lag_accuracy_dict, content_same_prioraccu_accuracy_dict)
    
    
    # extract questions data
    test_df = test_df.loc[test_df['content_type_id'] == 0]
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].astype('float32')
    
    
    # add and update dup_time features
    test_df, dup_time_matrix, user_accu_dict = update_dup_num(test_df, user_accu_dict, dup_time_matrix, user_master, content_master)
    
    # merge contents
    test_df = pd.concat([test_df.reset_index(drop=True), 
                        content_base[contents_use_col].reindex(test_df['content_id'].values).reset_index(drop=True)], axis=1)
    
    # preprocess(elapsed_timeが異常なユーザーをnp.nanにする)
    test_df.loc[test_df['prior_question_elapsed_time'] <= 1000, 'prior_question_elapsed_time'] = np.nan
            
    # category変数対応(null埋め)
    test_df['prior_question_had_explanation'].fillna(1, inplace = True)
    
    # debug用
    if check_test:
        all_test = all_test.append(test_df)
        
    # pred(NN)
    test_df['answered_correctly'] = (predict_MLP(test_df[use_col_mlp], mlp_params, mlp_model, mlp_ft, inplace=True) * 0.4)
        
    # pred(LGB)
    test_df['answered_correctly'] += (model.predict(test_df[use_col_lgb]) * 0.6)
    
    set_predict(test_df[['row_id', 'answered_correctly']])
    
    print('batch end:', round(time.time()-start, 3))

start batch
batch end: 0.838
batch end: 1.009
batch end: 1.186
batch end: 1.367
CPU times: user 849 ms, sys: 40.3 ms, total: 889 ms
Wall time: 1.37 s


## 推論結果デバッグ

In [29]:
# check_testの場合、推論結果をall_testオブジェクトで出力
# 内容を確認したいカラムを入力（trainと比較して一致してればOK）
if check_test:
    check_cols = use_col_mlp

    display(all_test[['user_id', 'timestamp'] + check_cols].sort_values(['user_id', 'timestamp']))